<a href="https://colab.research.google.com/github/naphtron/sagemaker-labs/blob/main/StreetSmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip install langchain langchain-community
# !pip install --quiet cohere
# !pip install unstructured[md]
# !pip install --upgrade chromadb==0.4.14
# !pip install --upgrade --quiet  langchain-google-genai pillow


In [6]:
import os
import shutil
import getpass

In [7]:
os.environ['COHERE_API_KEY'] = getpass.getpass('Cohere API Key:')

Cohere API Key:··········


In [8]:
import cohere
co = cohere.Client(os.environ['COHERE_API_KEY'])

In [9]:
# Define a custom Cohere embeddings class
class CohereEmbeddings:
    def __init__(self, model='embed-english-v2.0'):
        self.model = model

    def embed_documents(self, texts):
        response = co.embed(texts=texts, model=self.model)
        return response.embeddings
    def embed_query(self, query):
        return self.embed_documents([query])[0]

In [10]:
from langchain.vectorstores.chroma import Chroma

In [11]:
CHROMA_PATH = 'chroma_r'

In [12]:
embedding_function = CohereEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [13]:
query_text = "Is driving drunk illegal"

In [14]:
results = db.similarity_search_with_relevance_scores(query_text, k=3)

#result: List[Tuple[Document, float]]]
print(results)

[(Document(page_content='(1) Any person who, when driving or attempting to drive, or when in charge of a motor vehicle on a road or other public place, is under the influence of drink or a drug to such an extent as to be incapable of having proper control of the vehicle shall be guilty of an offence and liable to a fine', metadata={'source': 'data/The Traffic Act.md', 'start_index': 59877}), 0.557420551776886), (Document(page_content='44. Driving under influence of drink.', metadata={'source': 'data/The Traffic Act.md', 'start_index': 59838}), 0.5414082407951355), (Document(page_content='45. Prohibition of drinking when driving or in charge of public service vehicle.', metadata={'source': 'data/The Traffic Act.md', 'start_index': 60552}), 0.5388929843902588)]


In [21]:
PROMPT_TEMPLATE = """
You are an expert AI assistant. Answer the following question in detail, providing as much context and explanation as possible based on the provided information, while stating that it is from The Traffic Act of Kenya:

{context}

---

Given the above context, provide a detailed and comprehensive answer to the following question: {question}
"""

In [16]:
from langchain.prompts import ChatPromptTemplate

In [22]:
if len(results) == 0 or results[0][1] < 0.5:
    print(f"Unable to find matching results.")


context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)


Human: 
You are an expert AI assistant. Answer the following question in detail, providing as much context and explanation as possible based on the provided information, while stating that it is from The Traffic Act of Kenya:

(1) Any person who, when driving or attempting to drive, or when in charge of a motor vehicle on a road or other public place, is under the influence of drink or a drug to such an extent as to be incapable of having proper control of the vehicle shall be guilty of an offence and liable to a fine

---

44. Driving under influence of drink.

---

45. Prohibition of drinking when driving or in charge of public service vehicle.

---

Given the above context, provide a detailed and comprehensive answer to the following question: Is driving drunk illegal



In [18]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [23]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke(prompt)
print(result.content)

**Yes, driving drunk is illegal under The Traffic Act of Kenya.**

Section 44 of the Act states that any person who drives or attempts to drive a motor vehicle while under the influence of drink or drugs to an extent that they are incapable of having proper control of the vehicle is guilty of an offense. This means that drivers must not consume any amount of alcohol or drugs that would impair their ability to drive safely.

Additionally, Section 45 of the Act prohibits drivers of public service vehicles from drinking alcohol while driving or being in charge of the vehicle. This includes buses, taxis, and other vehicles used to transport passengers.


In [24]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00
